In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pymorphy2
from pyaspeller import YandexSpeller
from lru import LRU
import pickle
from time import sleep
from concurrent.futures import ThreadPoolExecutor
import tarfile
import bz2
import sys
import numpy as np
with open('ill_url_ids.pickle', 'rb') as file:
    ill_url_ids = pickle.load(file)
def transform(arg, ill_url_ids):
    if arg not in ill_url_ids:
        return arg
    else:
        return ill_url_ids[arg]
alpha = np.array([0.41,0.16,0.105,0.08,0.06,0.05,0.35,0.3,0.25,0.2])
class URL_info():
    def __init__(self):
        self.total_shows = 0
        self.total_clicks = 0
        self.num_first_clicks = 0
        self.num_last_clicks = 0
        self.show_poses = np.zeros((10,),dtype='int')
        self.show_poses_cl = np.zeros((10,),dtype='int')
        self.click_poses = np.zeros((10,),dtype='int')
        self.time = 0
        self.clicks_on_higher_pos = 0
        self.clicks_on_lower_pos = 0
        self.clicks_after_this = 0
        self.clicks_before_this = 0
        self.is_click_on_higher_pos = 0
        self.is_click_on_lower_pos = 0

In [2]:
df_marks = pd.read_csv('train.marks.tsv', delimiter='\t', header=None)
df_marks = df_marks.rename(columns={0: "QueryId", 1: "DocumentId"})
df_marks = df_marks.drop(columns=[2])
df_example = pd.read_csv('sample.csv')
all_groups = df_marks.append(df_example)
all_groups = all_groups.reset_index()
all_groups = all_groups.drop(columns=['index'])
all_groups = all_groups.sort_values(by=['QueryId', 'DocumentId']).reset_index()
all_groups = all_groups.drop(columns=['index'])
rev_frame = all_groups.sort_values(by=['DocumentId', 'QueryId']).reset_index()
rev_frame = rev_frame.drop(columns=['index'])

In [3]:
all_groups

,QueryId,DocumentId
0,0,1443
1,0,5912
2,0,5963
3,0,6096
4,0,6230
...,...,...
606045,6310,497350
606046,6310,512263
606047,6310,529857
606048,6310,551291


In [4]:
vals = all_groups['DocumentId'].values
qifs = all_groups['QueryId'].values

In [5]:
with open("doc_to_host.pickle", 'rb') as file:
    doc_to_host = pickle.load(file)
with open("QDP.pickle", 'rb') as file:
    url_simple = pickle.load(file)
with open("QHP.pickle", 'rb') as file:
    host_simple = pickle.load(file)

In [6]:
template = np.arange(1,11)
click_feats = dict()
click_feats1 = dict()
click_feats2 = dict()
click_feats3 = dict()
click_feats4 = dict()
click_feats['QD_fctr'] = []
click_feats['QD_lctr'] = []
click_feats['QD_avg_show_poses'] = []
click_feats['QD_avg_show_poses_cl'] = []
click_feats['QD_avg_click_poses'] = []

for i in range(1,11):
    click_feats['QD_show_poses_cl{}'.format(i)] = []

for i in range(1,11):
    click_feats2['QD_prob_cc{}'.format(i)] = []

click_feats['QH_fctr'] = []
click_feats['QH_lctr'] = []
click_feats['QH_avg_show_poses'] = []
click_feats['QH_avg_show_poses_cl'] = []
click_feats['QH_avg_click_poses'] = []

for i in range(1,11):
    click_feats1['QH_show_poses_cl{}'.format(i)] = []

for i in range(1,11):
    click_feats2['QH_prob_cc{}'.format(i)] = []

for i in range(1,11):
    click_feats3['QD_prob_sc{}'.format(i)] = []
    click_feats3['QH_prob_sc{}'.format(i)] = []
for i in range(1,11):
    click_feats4['QD_prob_ss{}'.format(i)] = []
    click_feats4['QH_prob_ss{}'.format(i)] = []

In [7]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    
    click_feats['QD_fctr'].append(url_simple[Q][key1].num_first_clicks/(url_simple[Q][key1].total_shows+0.0000001))
    click_feats['QD_lctr'].append(url_simple[Q][key1].num_last_clicks/(url_simple[Q][key1].total_shows+0.0000001))
    click_feats['QD_avg_show_poses'].append(np.sum(template*url_simple[Q][key1].show_poses)/(np.sum(url_simple[Q][key1].show_poses)+0.0000001))
    click_feats['QD_avg_show_poses_cl'].append(np.sum(template*url_simple[Q][key1].show_poses_cl)/(np.sum(url_simple[Q][key1].show_poses_cl)+0.0000001))
    click_feats['QD_avg_click_poses'].append(np.sum(template*url_simple[Q][key1].click_poses)/(np.sum(url_simple[Q][key1].click_poses)+0.0000001))
    
    for j in range(1,11):
        click_feats['QD_show_poses_cl{}'.format(j)].append(url_simple[Q][key1].show_poses_cl[j-1])

    for j in range(1,11):
        click_feats2['QD_prob_cc{}'.format(j)].append(url_simple[Q][key1].click_poses[j-1]/(url_simple[Q][key1].total_clicks+0.0000001))

    for i in range(1,11):
        click_feats3['QD_prob_sc{}'.format(i)].append(url_simple[Q][key1].show_poses_cl[i-1]/(url_simple[Q][key1].total_clicks+0.0000001))
    for i in range(1,11):
        click_feats4['QD_prob_ss{}'.format(i)].append(url_simple[Q][key1].show_poses[i-1]/(url_simple[Q][key1].total_shows+0.0000001))

100%|██████████| 606050/606050 [01:29<00:00, 6808.81it/s]


In [8]:
for i in tqdm(range(len(vals))):
    key = vals[i]
    Q = qifs[i]
    key1 = transform(key, ill_url_ids)
    key1 = doc_to_host[key1]
    
    click_feats['QH_fctr'].append(host_simple[Q][key1].num_first_clicks/(host_simple[Q][key1].total_shows+0.0000001))
    click_feats['QH_lctr'].append(host_simple[Q][key1].num_last_clicks/(host_simple[Q][key1].total_shows+0.0000001))
    click_feats['QH_avg_show_poses'].append(np.sum(template*host_simple[Q][key1].show_poses)/(np.sum(host_simple[Q][key1].show_poses)+0.0000001))
    click_feats['QH_avg_show_poses_cl'].append(np.sum(template*host_simple[Q][key1].show_poses_cl)/(np.sum(host_simple[Q][key1].show_poses_cl)+0.0000001))
    click_feats['QH_avg_click_poses'].append(np.sum(template*host_simple[Q][key1].click_poses)/(np.sum(host_simple[Q][key1].click_poses)+0.0000001))
    
    for j in range(1,11):
        click_feats1['QH_show_poses_cl{}'.format(j)].append(host_simple[Q][key1].show_poses_cl[j-1])

    for j in range(1,11):
        click_feats2['QH_prob_cc{}'.format(j)].append(host_simple[Q][key1].click_poses[j-1]/(host_simple[Q][key1].total_clicks+0.0000001))

    for i in range(1,11):
        click_feats3['QH_prob_sc{}'.format(i)].append(host_simple[Q][key1].show_poses_cl[i-1]/(host_simple[Q][key1].total_clicks+0.0000001))
    for i in range(1,11):
        click_feats4['QH_prob_ss{}'.format(i)].append(host_simple[Q][key1].show_poses[i-1]/(host_simple[Q][key1].total_shows+0.0000001))

100%|██████████| 606050/606050 [01:28<00:00, 6842.00it/s]


In [9]:
with open("click_feats_pair3.pickle", 'wb') as file:
    pickle.dump(click_feats, file)

In [10]:
with open("click_feats_pair4.pickle", 'wb') as file:
    pickle.dump(click_feats1, file)

In [11]:
with open("click_feats_pair5.pickle", 'wb') as file:
    pickle.dump(click_feats2, file)

In [12]:
with open("click_feats_pair6.pickle", 'wb') as file:
    pickle.dump(click_feats3, file)

In [13]:
with open("click_feats_pair7.pickle", 'wb') as file:
    pickle.dump(click_feats4, file)

In [14]:
8+9

17